<font color=blue><b>
Mandaram super bem, pessoal! Parabéns!!!
    
Vejam abaixo meus comentários :)
    
Nota: 9.8/10
</b></font>

_________

# SQUAD AZUL #

<font size=4>Lets Code - Projeto da disciplina de Machine Learning

<font size=4>Autores: Victor Cattani, Martin Lietz e Bruno Ary
    
08/10/21

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from skopt import gp_minimize
import warnings

warnings.filterwarnings("ignore")

path = os.getcwd()

df_train = pd.read_csv(path+"\\"+"credito-imoveis"+
                       "\\"+"application_train.csv", sep=",")

df_teste = pd.read_csv(path+"\\"+"credito-imoveis"+
                       "\\"+"application_test_student.csv", sep=",")


## Tratamento da Base de Dados e Feature Engineering

<font size=4>As próximas células apresentam processos associados ao tratamento da bases de dados e derivação de novos parâmetros. 

    



<font size=4> Em um primeiro momento, é feita a derivação de algumas variáveis, dentre elas:
    
    - Taxas relacionadas à receita de cada cliente.
    - Tempo empregado
    - Número de adultos e crianças na família
    - Parâmetros estatísticos associados às pontuações externas de cada cliente.
    - Número de documentos
    - Diferenciação dos dias de aplicação entre dias de semana e final de semana.

In [2]:
## FEATURE ENGINEERINNG

## taxas de receita --- Treinamento e Teste
df_train["credito_receita"] = df_train["AMT_CREDIT"]/df_train["AMT_INCOME_TOTAL"]
df_train["anuidade_receita"] = df_train["AMT_ANNUITY"]/df_train["AMT_INCOME_TOTAL"]
df_train["bens_receita"] = df_train["AMT_GOODS_PRICE"] / df_train["AMT_INCOME_TOTAL"]
df_train["receita_pessoa"] = df_train["AMT_INCOME_TOTAL"] / df_train["CNT_FAM_MEMBERS"]

df_teste["credito_receita"] = df_teste["AMT_CREDIT"]/df_teste["AMT_INCOME_TOTAL"]
df_teste["anuidade_receita"] = df_teste["AMT_ANNUITY"]/df_teste["AMT_INCOME_TOTAL"]
df_teste["bens_receita"] = df_teste["AMT_GOODS_PRICE"] / df_teste["AMT_INCOME_TOTAL"]
df_teste["receita_pessoa"] = df_teste["AMT_INCOME_TOTAL"] / df_teste["CNT_FAM_MEMBERS"]


# tempo percetual empregado -- Treinamento e Teste
df_train["tempo_empregado_percent"] = df_train["DAYS_EMPLOYED"] / df_train["DAYS_BIRTH"]
df_train["tempo_empregado_percent"][df_train["tempo_empregado_percent"] < 0] = None

df_teste["tempo_empregado_percent"] = df_teste["DAYS_EMPLOYED"] / df_teste["DAYS_BIRTH"]
df_teste["tempo_empregado_percent"][df_teste["tempo_empregado_percent"] < 0] = None


# número de adutlos --- Treinamento e Teste
df_train["num_adultos"] = df_train["CNT_FAM_MEMBERS"] - df_train["CNT_CHILDREN"]
df_train['tax_kid'] = df_train['CNT_CHILDREN'] / df_train['CNT_FAM_MEMBERS']

df_teste["num_adultos"] = df_teste["CNT_FAM_MEMBERS"] - df_teste["CNT_CHILDREN"]
df_teste['tax_kid'] = df_teste['CNT_CHILDREN'] / df_teste['CNT_FAM_MEMBERS']

# number of overall payments -- Treinamento e Teste
df_train['ANNUITY LENGTH'] = df_train['AMT_CREDIT'] / df_train['AMT_ANNUITY']

df_teste['ANNUITY LENGTH'] = df_train['AMT_CREDIT'] / df_train['AMT_ANNUITY']

# Recursos Externos -- Treinamento e Teste
df_train["EXT_SOURCE_MIN"]  = df_train[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].min(axis = 1)
df_train["EXT_SOURCE_MAX"]  = df_train[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].max(axis = 1)
df_train["EXT_SOURCE_MEAN"] = df_train[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].mean(axis = 1)
df_train["EXT_SOURCE_SD"]   = df_train[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].std(axis = 1)
df_train["NUM_EXT_SOURCES"] = 3 - (df_train["EXT_SOURCE_1"].isnull().astype(int) +
                               df_train["EXT_SOURCE_2"].isnull().astype(int) +
                               df_train["EXT_SOURCE_3"].isnull().astype(int))

df_teste["EXT_SOURCE_MIN"]  = df_teste[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].min(axis = 1)
df_teste["EXT_SOURCE_MAX"]  = df_teste[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].max(axis = 1)
df_teste["EXT_SOURCE_MEAN"] = df_teste[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].mean(axis = 1)
df_teste["EXT_SOURCE_SD"]   = df_teste[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].std(axis = 1)
df_teste["NUM_EXT_SOURCES"] = 3 - (df_teste["EXT_SOURCE_1"].isnull().astype(int) +
                               df_teste["EXT_SOURCE_2"].isnull().astype(int) +
                               df_teste["EXT_SOURCE_3"].isnull().astype(int))

# Número de documentos -- Treinamento e Teste
doc_vars = ["FLAG_DOCUMENT_2",  "FLAG_DOCUMENT_3",  "FLAG_DOCUMENT_4",  "FLAG_DOCUMENT_5",  "FLAG_DOCUMENT_6",
            "FLAG_DOCUMENT_7",  "FLAG_DOCUMENT_8",  "FLAG_DOCUMENT_9",  "FLAG_DOCUMENT_10", "FLAG_DOCUMENT_11",
            "FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13", "FLAG_DOCUMENT_14", "FLAG_DOCUMENT_15", "FLAG_DOCUMENT_16",
            "FLAG_DOCUMENT_17", "FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19", "FLAG_DOCUMENT_20", "FLAG_DOCUMENT_21"]
df_train["NUM_DOCUMENTS"] = df_train[doc_vars].sum(axis = 1)

df_teste["NUM_DOCUMENTS"] = df_teste[doc_vars].sum(axis = 1)

# Data De aplicação -- Treinamento e Teste
df_train["DAY_APPR_PROCESS_START"] = "Working day"
df_train["DAY_APPR_PROCESS_START"][(df_train["WEEKDAY_APPR_PROCESS_START"] == "SATURDAY") |
                               (df_train["WEEKDAY_APPR_PROCESS_START"] == "SUNDAY")] = "Weekend"

df_teste["DAY_APPR_PROCESS_START"] = "Working day"
df_teste["DAY_APPR_PROCESS_START"][(df_teste["WEEKDAY_APPR_PROCESS_START"] == "SATURDAY") |
                               (df_teste["WEEKDAY_APPR_PROCESS_START"] == "SUNDAY")] = "Weekend"

df_train["DAY_APPR_PROCESS_START"].astype('category')
df_teste["DAY_APPR_PROCESS_START"].astype('category')

# age ratios
df_train["OWN_CAR_AGE_RATIO"] = df_train["OWN_CAR_AGE"] / df_train["DAYS_BIRTH"]
df_train["DAYS_ID_PUBLISHED_RATIO"] = df_train["DAYS_ID_PUBLISH"] / df_train["DAYS_BIRTH"]
df_train["DAYS_REGISTRATION_RATIO"] = df_train["DAYS_REGISTRATION"] / df_train["DAYS_BIRTH"]

df_teste["OWN_CAR_AGE_RATIO"] = df_teste["OWN_CAR_AGE"] / df_teste["DAYS_BIRTH"]
df_teste["DAYS_ID_PUBLISHED_RATIO"] = df_teste["DAYS_ID_PUBLISH"] / df_teste["DAYS_BIRTH"]
df_teste["DAYS_REGISTRATION_RATIO"] = df_teste["DAYS_REGISTRATION"] / df_teste["DAYS_BIRTH"]

<font color=blue><b>
Isso ae, galera! Curti bastaaante o trabalho de feature engineering que vcs fizeram!
    
E parabéns por já terem criado as features na base de teste também -- isso é fundamental! Sem isso, naturalmente o modelo não funcionaria na base de teste!

<br>
Como vcs fizeram está perfeito! Eu apenas sugeriria que vocês colocassem tudo em uma função genérica, que cria as novas features pra qualquer df, e aí vcs aplicam esta mesma função às bases de treino e teste separadamente. Assim, fica mais com cara de uma Pipeline ;)
    
E, falando em Pipeline, também é possível adicionar o processo de feature engineering à uma Pipeline propriamente dita, o que garante ainda maior robustez! Para isso, sugiro que vcs deem uma olhadinha no [FeatureUnion](https://scikit-learn.org/stable/modules/compose.html#feature-union)
</b></font>

In [3]:
df_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246008 entries, 0 to 246007
Data columns (total 140 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    SK_ID_CURR                    int64  
 1    TARGET                        int64  
 2    NAME_CONTRACT_TYPE            object 
 3    CODE_GENDER                   object 
 4    FLAG_OWN_CAR                  object 
 5    FLAG_OWN_REALTY               object 
 6    CNT_CHILDREN                  int64  
 7    AMT_INCOME_TOTAL              float64
 8    AMT_CREDIT                    float64
 9    AMT_ANNUITY                   float64
 10   AMT_GOODS_PRICE               float64
 11   NAME_TYPE_SUITE               object 
 12   NAME_INCOME_TYPE              object 
 13   NAME_EDUCATION_TYPE           object 
 14   NAME_FAMILY_STATUS            object 
 15   NAME_HOUSING_TYPE             object 
 16   REGION_POPULATION_RELATIVE    float64
 17   DAYS_BIRTH                    int64  
 18   DA

In [4]:
df_teste.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61503 entries, 0 to 61502
Data columns (total 139 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    SK_ID_CURR                    int64  
 1    NAME_CONTRACT_TYPE            object 
 2    CODE_GENDER                   object 
 3    FLAG_OWN_CAR                  object 
 4    FLAG_OWN_REALTY               object 
 5    CNT_CHILDREN                  int64  
 6    AMT_INCOME_TOTAL              float64
 7    AMT_CREDIT                    float64
 8    AMT_ANNUITY                   float64
 9    AMT_GOODS_PRICE               float64
 10   NAME_TYPE_SUITE               object 
 11   NAME_INCOME_TYPE              object 
 12   NAME_EDUCATION_TYPE           object 
 13   NAME_FAMILY_STATUS            object 
 14   NAME_HOUSING_TYPE             object 
 15   REGION_POPULATION_RELATIVE    float64
 16   DAYS_BIRTH                    int64  
 17   DAYS_EMPLOYED                 int64  
 18   DAYS

<font size=4> Após a derivação de algumas variáveis, verificou-se a porcentagem entre de dados faltantes. Da tabela abaixo,nota-se que há um hiato significativo entre os valores pecentuais de 50 e 30. Portanto, decidiu-se que se excluíram somentes os parâmetros que teriam valores percentuais acima de 35%,

In [5]:
## Tratamento dos dados missing

total = df_train.isnull().sum().sort_values(ascending = False)

percent = (df_train.isnull().sum()/df_train.isnull().count()*100).sort_values(ascending=False)

missing_application_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_application_train_data=missing_application_train_data[missing_application_train_data['Percent']>0]

missing_application_train_data[missing_application_train_data["Percent"]>=10]

,Total,Percent
COMMONAREA_MODE,171978,69.907483
COMMONAREA_AVG,171978,69.907483
COMMONAREA_MEDI,171978,69.907483
NONLIVINGAPARTMENTS_MODE,170914,69.474976
NONLIVINGAPARTMENTS_MEDI,170914,69.474976
NONLIVINGAPARTMENTS_AVG,170914,69.474976
FONDKAPREMONT_MODE,168392,68.449807
LIVINGAPARTMENTS_AVG,168278,68.403467
LIVINGAPARTMENTS_MODE,168278,68.403467
LIVINGAPARTMENTS_MEDI,168278,68.403467


In [6]:
drops=list(missing_application_train_data[missing_application_train_data["Percent"]>=35].index)

df_train = df_train.drop(columns = drops)

df_teste = df_teste.drop(columns = drops)

<font color=blue><b>
Gostei bastante da forma como vcs estruturam a aferição de dados nulos, bem legal!!
    
E achei justo o critério de vocês pra dropar as colunas com mais do que certa porcentagem de dados nulos! De maneira geral, é um procedimento justo.
    
<br>
Dito isso, queria apenas fazer um comentário: lembrem-se que em alguns casos, a falta de informação também é informação! Por exemplo, a coluna "OWN_CAR_AGE". Naturalmente, as pessoas que não têm carro, terão NaN nesta coluna, certo? Pro outro lado, esta pode ser uma informação valiosa para as pessoas QUE TÊM carro. Por este motivo, pode não ser uma boa ideia jogar fora esta coluna. Poderíamos, ao invés, preencher os dados que estão faltando com -1, por exemplo (isso faz sentido, pois preenchendo com um valor negativo uma coluna que só deveria ter valores positivos, faz com que todos os NaNs sejam tratados da mesma maneira, e diferente de qualquer outro valor na base!).

Enfim, a mensagem que quero passar é: antes de simplesmente jogarmos fora uma coluna, vale a pena refletir um pouco antes pra saber se é justo que joguemos tudo fora ou não, legal?? Por isso, é fundamental uma EDA bem feita para que conheçamos 100% a base e suas colunas, assim podemos saber com segurança o que jogar fora e o que não! (E, infelizmente, senti falta disso no trabalho de vcs)
    
Isso, claro, pensando em fazer modelos muuuito performáticos, e tentando garantir que fizemos o melhor que pode ser feito, nos aproveitando de toda a info disponpivel. Ou seja, seria apenas um algo a mais a se tentar, caso vocês no final não estivessem satisfeitos ainda com a performance do modelo! ;)
</b></font>

<font size =4> As variáveis categóricas, como de prache, foram transformadas em numéricas através de uma lógica semelhante do OneHoteEnconding. As variáveis que possuem somente dois únicos foi feito um Label Enconder. As demais variáveis passaram por um processo que será exposto posteriormente,

In [7]:
categorical = [col for col in df_train.columns if df_train[col].dtypes == 'object']
ct = df_train[categorical].nunique().sort_values()

ct

NAME_CONTRACT_TYPE             2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
DAY_APPR_PROCESS_START         2
CODE_GENDER                    3
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
NAME_TYPE_SUITE                7
WEEKDAY_APPR_PROCESS_START     7
NAME_INCOME_TYPE               8
OCCUPATION_TYPE               18
ORGANIZATION_TYPE             58
dtype: int64

In [8]:
lb = LabelEncoder()

for col in categorical:
    if len(df_train[col].unique()) ==2:
        lb.fit(df_train[col])
        
        print(col)
        
        df_train["o"+col] = lb.transform(df_train[col])
        
        df_teste["o"+col] = lb.transform(df_teste[col])
           
df_train

NAME_CONTRACT_TYPE
FLAG_OWN_CAR
FLAG_OWN_REALTY
DAY_APPR_PROCESS_START


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,EXT_SOURCE_SD,NUM_EXT_SOURCES,NUM_DOCUMENTS,DAY_APPR_PROCESS_START,DAYS_ID_PUBLISHED_RATIO,DAYS_REGISTRATION_RATIO,oNAME_CONTRACT_TYPE,oFLAG_OWN_CAR,oFLAG_OWN_REALTY,oDAY_APPR_PROCESS_START
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,...,0.373285,2,1,Working day,0.157100,0.479829,0,0,0,1
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,...,0.286485,3,1,Working day,0.241994,0.261651,0,0,0,1
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,...,0.055561,2,1,Working day,0.298363,0.569642,0,1,0,1
3,361264,0,Cash loans,F,N,Y,0,270000.0,814041.0,28971.0,...,0.060841,2,1,Working day,0.077235,0.526421,0,0,1,1
4,260639,0,Cash loans,F,N,Y,0,144000.0,675000.0,21906.0,...,0.192295,3,1,Working day,0.126387,0.269069,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246003,242114,0,Cash loans,F,N,Y,1,270000.0,1172470.5,34411.5,...,0.008268,2,1,Working day,0.255690,0.626813,0,0,1,1
246004,452374,0,Cash loans,F,N,Y,0,180000.0,654498.0,27859.5,...,0.248636,2,1,Weekend,0.194848,0.156904,0,0,1,0
246005,276545,1,Revolving loans,M,N,N,1,112500.0,270000.0,13500.0,...,0.054030,2,0,Weekend,0.301021,0.515717,1,0,0,0
246006,236776,1,Cash loans,M,Y,N,3,202500.0,204858.0,17653.5,...,0.299722,2,1,Weekend,0.308156,0.131780,0,1,0,0


<font color=blue><b>
Legal! Aqui, novamente, vocês fizeram o mesmo tratamento pra base de treino e de teste, então está certinho.
    
Mas lembrem-se do Pipeline! É mais prático, rápido e seguro colocar essa e as demais etapas de pré-processamento dentro do Pipeline, pois assim garantimos que não vai ter problema algum de data leakage.
    
Mas, de fato, isso só vai se tornar algo notavelmente importante em modelos produtivos (no último módulo do curso vcs vão entender pq). Enquanto não estamos pensando em modelos produtivos, o que vocês fizeram tá certinho!
</b></font>

<font size=4> Das variáveis categóricas que possuem o maior número de valores únicos (OCCUPATION_TYPE E ORGANIZATION_TYPE), decidimos tratá-las de maneira especial. A partir do valor médio dos target fizemos um clusterização de modo que os valores únicos para as variáveis categóricas fossem reduzidos e, desta forma, não onerasse (pelo número excessivo de features) os modelos durante o treinamento e predição. Para determinar o número ideal de cluster utilizamos o método da silhueta.

In [9]:
organ_sort = df_train.groupby(['ORGANIZATION_TYPE'])['TARGET'].agg(['mean'])

occupation_sort = df_train.groupby(["OCCUPATION_TYPE"])['TARGET'].agg(['mean'])

categorical_df_lista=[organ_sort, occupation_sort]

lista_k = list(range(2,10))

for df in categorical_df_lista:
    
    silhouete_scores =[]
    
    for k in lista_k:
    
        estimador = KMeans (n_clusters=k, max_iter=1000)

        modelo = estimador.fit(df) 

        silhouete_scores.append(silhouette_score(df, modelo.labels_))

    k=lista_k[np.argmax(silhouete_scores)]

    estimador = KMeans(n_clusters=k, max_iter=1000,random_state=2)

    modelo = estimador.fit(df)

    labels = modelo.predict(df)
    
    df=df.reset_index()

    dados_cluster = pd.concat([df, 
                                 pd.Series(labels, name="cluster_"+df.columns[0])], axis=1)
    
    df_train=df_train.merge(dados_cluster[[dados_cluster.columns[0],"cluster_"+df.columns[0]]],on=dados_cluster.columns[0], how='left')
    
    df_teste=df_teste.merge(dados_cluster[[dados_cluster.columns[0],"cluster_"+df.columns[0]]],on=dados_cluster.columns[0], how='left')

    df_train["cluster_"+df.columns[0]] = df_train["cluster_"+df.columns[0]].astype('category')
    
    df_teste["cluster_"+df.columns[0]] = df_teste["cluster_"+df.columns[0]].astype('category')

<font color=blue><b>
Uaaau, aí sim hein!! Achei <i>FANCY</i> hahahaha
    
Mas, sério, parabéns por terem usado uma técnica de clusterização como pré-processamento! Isso demonstra uma compreensão muuuito boa de tudo o que aprendemos no curso, e utilização das técnicas em contextos bem específicos, o que evidencia que vcs absorveram super bem a essência do que aprendemos!. Gostei muuuito, parabéns mesmo!!
</b></font>

<font size=4> Nesta etapa as variáveis categóricas que já sofreram o processo Encoding e as clusterizadas são dropadas da base, e as demais variáveis categóricas são transformadas em dummies.

In [10]:
discard_features = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_CONTRACT_TYPE', 'DAY_APPR_PROCESS_START']

df_train.drop(discard_features,axis = 1, inplace = True)

df_teste.drop(discard_features,axis = 1, inplace = True)

df_train = pd.get_dummies(df_train)

df_teste = pd.get_dummies(df_teste)

train_labels = df_train['TARGET']
df_train, df_teste = df_train.align(df_teste, join = 'inner', axis = 1)
df_train['TARGET'] = train_labels


<font size=4> Por fim, é feito o processo de normalização e preenchimento dos valores faltantes. No primeiro caso os dados foram normalizados entre 0 e 1, e os dados faltantes foram preenchidas pela mediana dos parâmetros.

In [11]:
## Fill in missing values

from sklearn.impute import SimpleImputer

imputer= SimpleImputer(strategy='median')
scaler = MinMaxScaler(feature_range = [0,1])
train = df_train.drop(columns = ['TARGET'])

imputer.fit(train)
train = imputer.transform(train)
test = imputer.transform(df_teste)
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)


<font color=blue><b>
Mais uma vez, muito bem! Mandaram bem demais!!
    
Aqui, faço novamente o comentário sobre a utilização do Pipeline (ressaltando, novamente, que apesar do comentário, tá certinho como vcs fizeram!)
</b></font>

## Aplicação das técnicas de aprendizado de máquina

<font size=4> Após o processo de feature engineering, forama aplicados algumas técnicas de aprendizado de máquina. Em se tratando de um problema de classificação e o grau de complexidade do modelo, foram utilizados os seguintes modelos:
    
    -Regressão Logística
    -Random Forest
    -LightGBM
    
O LightGBM é uma primo próximo ao XGBoost, portanto, funciona como sendo um ensemble de árvores. Contudo, o seu processo de aprendizado é mais rápido para alguns casos. Uma explicação mais detalhada do modelo está [nesse link.](https://datarisk.io/gradient-boostings-parte-3-xgboost-vs-lightgbm-vs-catboost/?cn-reloaded=1).  

<font color=blue><b>
Perfeito!! Não chegamos a falar em detalhes sobre o LightGBM, então fico muuito feliz de vcs terem ido atrás dele por conta própria! Isso mostra que vocês estão agindo mto bem de maneira autodidata e independente (e exercitando mto bem a importante habilidade de pesquisa hehe), então, parabéns demais!!
</b></font>

### ----------------------------------------------Regressão Logística ------------------------------------------- ###


<font size=4>Para o algoritmo de regressão logística, o processamento de determinação dos hiperparâmetros do modelo deu-se através do GridSearchCV. Desta forma, todas as combinações possíveis para os valores dos hiperparâmetros são testados. Neste caso, as regularizações lasso e ridge ridge regression foram utilizadas bem como diferentes valores de trade-off. 

In [96]:
### Regressão Logística ### 
param_grid_reg_log = {'C' : [0.01,0.1,1,10,100],
              'penalty' : ['l1','l2']}
log_reg = LogisticRegression()
grid_search_reg_log = GridSearchCV(log_reg, param_grid_reg_log, scoring = 'roc_auc', cv = 5, verbose=1.5)
grid_search_reg_log.fit(train, train_labels)

# Train on the training data
log_reg_best = grid_search_reg_log.best_estimator_
log_reg_pred = log_reg_best.predict_proba(test)[:, 1]
submit = df_teste[['SK_ID_CURR']]
submit['REGRESSÃO_LOGISTICA'] = log_reg_pred

grid_search_reg_log.best_score_


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END .................................C=0.01, penalty=l1; total time=   4.5s
[CV] END .................................C=0.01, penalty=l1; total time=   0.1s
[CV] END .................................C=0.01, penalty=l1; total time=   0.1s
[CV] END .................................C=0.01, penalty=l1; total time=   0.1s
[CV] END .................................C=0.01, penalty=l1; total time=   0.1s
[CV] END .................................C=0.01, penalty=l2; total time=   8.5s
[CV] END .................................C=0.01, penalty=l2; total time=   8.4s
[CV] END .................................C=0.01, penalty=l2; total time=   8.4s
[CV] END .................................C=0.01, penalty=l2; total time=   8.3s
[CV] END .................................C=0.01, penalty=l2; total time=   8.7s
[CV] END ..................................C=0.1, penalty=l1; total time=   0.1s
[CV] END ..................................C=0.1

0.7466520076364109

<font color=blue><b>
Ótimo!!
</b></font>

### ------------------------------- Random Forest ------------------------------------------- ###

<font size=4> No que diz respeito à tecnica de random forest, a definição dos hiperparâmetros deu-se também através do GridSearch. Neste caso, foram utilizados os seguintes parâmetros:
    
**Critério**: Determina qual é o melhor critério que determina o melhor ganho de informação no momento da separação (entropia ou gini).
 
**Máxima profundidade**: Como o próprio nome sugere, determina a máxima profundidade de das árvores que compõem o ennemble. Utiizou-se um intervalo entre 2 e 5.
    
**Máximo número de  features**: Determina qual operação aritmética será utilizada para selecionar o número máximo de parâmetros no momento da separação da árvore. 
    
**Número Estimadores**: Determina o número de árvores utilizado pelo ensemble. Neste caso, utilizou-se 50,100,150,200.

In [93]:
## Random Forest ###

param_grid_reg_rf = {'criterion' : ["gini","entropy"], 
                     "max_depth":[i for i in range(2,5)], 
                     "max_features":["sqrt", "log2"], 
                     "n_estimators" : [50,100,150,200]}

rf = RandomForestClassifier(random_state=42)

grid_search_rf = GridSearchCV(rf, param_grid_reg_rf, scoring = 'roc_auc', cv = 5, verbose=1.5)
grid_search_rf.fit(train, train_labels)

# Train on the training data
rf_best = grid_search_rf.best_estimator_
rf_pred = rf_best.predict_proba(test)[:, 1]
submit = df_teste[['SK_ID_CURR']]


grid_search_rf.best_score_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=50; total time=  12.6s
[CV] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=50; total time=  11.8s
[CV] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=50; total time=  10.9s
[CV] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=50; total time=   9.8s
[CV] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=50; total time=   8.5s
[CV] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=100; total time=  15.1s
[CV] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=100; total time=  15.2s
[CV] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=100; total time=  15.9s
[CV] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=100; total time=  15.3s
[CV] END criterion=gini, max_depth=2, max_features=sqrt, n_estimators=100; total time=  14.8s
[CV

[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=100; total time=  26.9s
[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=100; total time=  28.3s
[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=100; total time=  28.8s
[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=150; total time=  41.4s
[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=150; total time=  41.7s
[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=150; total time=  41.2s
[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=150; total time=  41.1s
[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=150; total time=  47.2s
[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=200; total time= 1.0min
[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=200; total time=  55.4s
[CV] END criterion=gini, max_depth=4, max_features=sqrt, n_e

[CV] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=150; total time=  37.6s
[CV] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=150; total time=  35.8s
[CV] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=200; total time=  48.1s
[CV] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=200; total time=  52.8s
[CV] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=200; total time=  53.8s
[CV] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=200; total time=  48.0s
[CV] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=200; total time=  48.9s
[CV] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=50; total time=  15.9s
[CV] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=50; total time=  14.9s
[CV] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=50; total time=  11.0s
[CV] END criterion=entropy, max_d

0.7340532292336415

<font color=blue><b>
Apenas um pequeno comentário: modelos random forest tendem a funcionar melhor com um grande número de estimadores (da ordem de 1000), o que faz sentido, dado o procedimento de bagging que é utilizado, né?
    
Fica como sugestão para próximos projetos: apesar de demorar beeeem mais pra treinar, pode valer a pena testar um número maior de estimadores também! (E aliás, foi provavelmente por este motivo que o random forest não teve uma performance tão legal aqui!)
</b></font>

In [97]:
submit['RANDOM_FOREST'] = rf_pred

### ------------------------------- Light GBM------------------------------------------- ###

<font size=4> Por fim, a determinação dos hiperparâmetros para a técnica Light GBM deu-se através de um processo de otimização. Este baseia-se em um processo de otimização Bayesiana usando processos Gaussianos [mais informações aqui]((https://datarisk.io/gradient-boostings-parte-3-xgboost-vs-lightgbm-vs-catboost/?cn-reloaded=1)). Assim sendo, o processo de otimização tem como variáveis de decisão os intervalos de valores estabelecidos para os hiperparâmetros do modelo e como função obejtivo a equação analítica (1 - ROC_SCORE). Utilizou-se esta uma vez que o processo de otimização tem como pressuposto atingir o mínimo valor possível  para a  função objetivo. Neste caso, assim como exposto para as demais técnicas previamente expostas, estamos tentando achar o valor máximo para a métrica ROC, assim, a minimização da equação analítica indica a maximização da métrica ROC. Dentre os hiperparâmetros, foram utilizados a taxa aprendizado, termo de regularização para o L1 e L2, número de estimadores, máxima profunidade das árvores, entre outros. Salienta-se que o valor utilizado de ROC junto à função objetivo é dado como a média entre os valores obtidos de roc, que é obtido a partir de um processo de validação cruzada. 

In [72]:
from sklearn.metrics import roc_auc_score
from skopt.space import Integer
from skopt.space import Real

def param_search (params_range):
    
    learning_rate =params_range[0],
    num_leaves = params_range[1],
    min_child_samples = params_range[2]
    n_estimators=params_range[3]
    max_depth = params_range[4]
    reg_alpha=params_range[5]
    subsample=params_range[6]
    colsample_bytree=params_range[7]
    reg_lambda = params_range[8]
    min_child_weight = params_range[9]
    min_split_gain = params_range[10]
   
    
    mdl = lgb.LGBMClassifier(learning_rate =learning_rate,
                             num_leaves = num_leaves,
                             min_child_samples = min_child_samples,
                             n_estimators=n_estimators,
                             max_depth = max_depth,
                             reg_alpha = reg_alpha,
                             random_state=42,
                             subsample=subsample,
                             colsample_bytree=colsample_bytree,
                             reg_lambda = reg_lambda,
                             min_child_weight = min_child_weight,
                             min_split_gain = min_split_gain)
    
    #mdl.fit(X_train,Y_train, eval_set=(X_val, Y_val), 
            #eval_metric= 'auc',early_stopping_rounds= 200, verbose=1.5) 
        
    result = cross_val_score (mdl, train, train_labels, scoring = 'roc_auc', cv=5, verbose=1.5)
    
    estimate = np.mean(result)
    
    print(estimate)
                            
    return 1-estimate

params_range =[
    (0.0001, 0.3,'log-uniform'), ## 0-learning_rate,
    (2,1000), ## 1-num_leaves
    (1,100), ## 2-min_child_samples
    (50,10000), ##3-n_estimators
    (-1,20), ## 4-max_depth
    Real(0.01,10), ## 5-reg_alpha
    Real(0.01,1), ##6-sub_sample
    Real (0.01,1), ##7-colsample_bytree,
    Real(0.01,10), ## 8 - reg_lambda
    (0.01,50), ## 9 - min_child_weight,
    (0.01,50)] ## 10 - min_split_gain 
  
result_gp=gp_minimize (param_search, params_range, random_state = 42, verbose=True, n_calls=5,  
                             n_random_starts=3)
    

print("""Best parameters:
            - 0) learning_rate=%f
            - 1) num_leaves=%f
            - 2) min_child_samples=%f
            - 3) n_estimators=%f
            - 4) max_depth=%f
            - 5) reg_alpha = %f
            - 6) sub_sample = %f
            - 7) colsample_bytree = %f
            - 8) reg_lambda = %f 
            - 9) min_child_weight =%f
            - 10) min_split_gain =%f""" %(result_gp.x[0],
                                   result_gp.x[1],
                                   result_gp.x[2],
                                   result_gp.x[3],
                                   result_gp.x[4],
                                   result_gp.x[5],
                                   result_gp.x[6],
                                   result_gp.x[7]),
                                   result_gp.x[8]),
                                   result_gp.x[9],
                                   result_gp.x[10]))



Iteration No: 1 started. Evaluating function at random point.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 1.6min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.6min remaining:    0.0s


[CV] END .................................................... total time= 1.8min
[CV] END .................................................... total time= 1.6min
[CV] END .................................................... total time= 1.6min
[CV] END .................................................... total time= 1.7min
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 497.8520
Function value obtained: 0.2350
Current minimum: 0.2350
Iteration No: 2 started. Evaluating function at random point.


...

[CV] END .................................................... total time= 2.8min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.8min remaining:    0.0s


[CV] END .................................................... total time= 2.1min
[CV] END .................................................... total time= 1.9min
[CV] END .................................................... total time= 2.1min
[CV] END .................................................... total time= 2.0min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 10.8min finished


Iteration No: 10 ended. Search finished for the next optimal point.
Time taken: 648.8571
Function value obtained: 0.2387
Current minimum: 0.2343
Best parameters:
            - 0) learning_rate=0.001784
            - 1) num_leaves=339
            - 2) min_child_samples=61
            - 3) n_estimators=9636.000000
            - 4) max_depth=5.000000
            - 5) reg_alpha = 5
            - 6) sub_sample = 0
            - 7) colsample_bytree = 0
            - 8) reg_lambda = 7 
            - 9) min_child_weight =6
            - 10) min_split_gain =0


<font color=blue><b>
Mais uma vez, UAAAAU!!
    
Embora eu tenha mencionado sobre otimização baeysiana em algumas aulas bem en passant, nós não abordamos estes métodos em detalhes pq eles são tecnicamente bem complicados de se entender (imagino que vocês tenham sentido isso quando procurarm pelo método, não é mesmo? rs). Por isso, esse acaba sendo um método de otimização de hiperparâmetros raramente abordado em um primeiro curso, por ser uma técnica bem avançada. Então, fiquei muuuuuito feliz em ter visto que vocês foram atrás e aplicaram esse método, muuuuito bem, de verdade!! Está mais do que claro que vcs não somente absorveram mto bem tudo o que aprendemos no curso, como também desenvolveram super bem a importante habilidade de pesquisa! Parabéns mesmo pessoal, continuem assim!
</b></font>

<font size=4> Com os hiperparâmetros obtidos, estes são adicionados ao modelo para o treinamento do modelo (O atributo .x retorna uma lista com os valores dos hiperparâmetros que resultaram no menor valor da função objetivo)  . 

In [ ]:
### Treinamento do modelo com os hiperparâmetros obtidos na otimização. 

mdl = lgb.LGBMClassifier(learning_rate =result_gp.x[0],
                             num_leaves = result_gp.x[1],
                             min_child_samples = result_gp.x[2],
                             n_estimators=result_gp.x[3],
                             max_depth = result_gp.x[4],
                             random_state=42,
                             reg_alpha = result_gp.x[5],
                             subsample=result_gp.x[6],
                             colsample_bytree=result_gp.x[7],
                             reg_lambda = result_gp.x[8],
                             min_child_weight = result_gp.x[9],
                             min_split_gain = result_gp.x[10])

mdl.fit(train,train_labels, eval_set=(train, train_labels),
            eval_metric= 'auc',early_stopping_rounds= 200, verbose=1.5) 
    
mdl.best_score_

Training until validation scores don't improve for 200 rounds


<font size =4> Aplicação da modelagem junto aos dados de teste

In [90]:
lgbm_pred=mdl.predict_proba(test)[:,1]

submit["LGBM_PRED"] = lgbm_pred

In [91]:
submit

,SK_ID_CURR,PROB_ABORDAGEM_2_RANDOM_FOREST,LGBM_PRED
0,149741,0.225538,0.185502
1,363290,0.033843,0.042619
2,436006,0.094906,0.093070
3,377703,0.051528,0.046153
4,188624,0.321391,0.229862
...,...,...,...
61498,102817,0.164420,0.099415
61499,343961,0.110765,0.082015
61500,427828,0.023412,0.033431
61501,405956,0.083919,0.032192


In [95]:
submit.to_csv("submit_squad_azul_2.csv", sep=";", index=False)

## Conclusões ##

- A técnica de regressão logística por se tratar de um problema de classificação linear apresentou resultados surpreendemente bons. 

- O mau desempenho do Random Forest em relação aos demais possivelmente está atrelado às definições do hiperparâmetros, necessitando de uma análise mais detalhada nesse sentido.

- A técnica de LightGBm por se mais poderosa em relação às demais apresentou um valor de roc visivlemente superior.

- O processo de feature engineering foi fundamental no bom desempenho das técnicas de ML utilizadas. O bom desempenho neste caso se baseia nos resultados obtidos por outros programadores para o problema. 

<font color=blue><b>
Ótimas conclusões, pessoal! Todas corretas, muito bem observado! (Lembrando que o lightgbm tbm usou otimização bayesiana, então tem isso tbm hehehe).
 
Pelos meus comentários no meio do código, dá pra ver que fique bastante feliz coma solução de vcs, né? Vcs mandaram muito bem mesmo, parabéns!
   
<br>
Além dos comentários pontuais supracitados, a única coisa que eu senti falta foi de uma análise exploratória. Vocês já começaram o notebook com o processo de feature engineering (talvez inspirados em algumas outras soluções que vcs viram, como vcs disseram (e tá tudo certo!!)), mas notem que na vida real dificilmente será possível fazer feature engineering sem que vcs antes conheçam muuuuuito bem o problema e a base de vcs, o que só é possível com extensa EDA. E, bom, eu imagino que vocês tenham sim feito alguma EDA, apenas não colocaram aqui no notebook final, hehe. De qualquer maneira, fica essa dica e o lembrete de que este tempo que passamos explorando a base é de enooooorme importância e necessidade, como fizemos extensivamente no módulo 3 hehe.
    
Enfim, de maneira geral, curti bastante o trabalho de vcs! Vcs certamente mostraram ter absorvido muuuito bem a essência de tudo o que estudamos nestes meses, então fico bastante feliz! :)
</b></font>

<font color=blue><b>
___________
    
Eu enviei pra vocês o arquivo `test_targets.csv`, que contém o target real da base de teste. Vou usá-lo abaixo pra produzir a roc_auc de teste de vcs:
</b></font>

In [1]:
import pandas as pd

In [2]:
df_test = pd.read_csv("test_targets.csv")

df_test.head()

,SK_ID_CURR,TARGET
0,149741,1
1,363290,0
2,436006,0
3,377703,0
4,188624,0


In [4]:
df_pred = pd.read_csv("submit_squad_azul.csv", sep=";")

df_pred.head()

,SK_ID_CURR,REGRESSÃO_LOGISTICA,RANDOM_FOREST,LGBM_PRED
0,149741,0.225538,0.147858,0.185502
1,363290,0.033843,0.052104,0.042619
2,436006,0.094906,0.116577,0.093070
3,377703,0.051528,0.062726,0.046153
4,188624,0.321391,0.159345,0.229862


In [5]:
df_merge = df_pred.merge(df_test, on="SK_ID_CURR", how="inner")

df_merge.head()

,SK_ID_CURR,REGRESSÃO_LOGISTICA,RANDOM_FOREST,LGBM_PRED,TARGET
0,149741,0.225538,0.147858,0.185502,1
1,363290,0.033843,0.052104,0.042619,0
2,436006,0.094906,0.116577,0.093070,0
3,377703,0.051528,0.062726,0.046153,0
4,188624,0.321391,0.159345,0.229862,0


In [19]:
from sklearn.metrics import roc_auc_score

y_true = df_merge["TARGET"]

for col in ['REGRESSÃO_LOGISTICA', 'RANDOM_FOREST', 'LGBM_PRED']:
    
    y_score = df_merge[col]

    print(f"ROC-AUC de teste para o modelo {col}: {roc_auc_score(y_true, y_score):.4f}\n")

ROC-AUC de teste para o modelo REGRESSÃO_LOGISTICA: 0.7448

ROC-AUC de teste para o modelo RANDOM_FOREST: 0.7315

ROC-AUC de teste para o modelo LGBM_PRED: 0.7468



<font color=blue><b>
Boaaa, esses são resultados beeeem legais, parabéns!! De fato, o trabalho de feature engineering que vcs fizeram certamente contribuiu bastante pra esse resultado legal -- o que corrobora mais uma vez como o processo de EDA/data wrangling é FUNDAMENTAL para um bom resultado em projetos de data science!!
    
De fato, é muito legal ver que o logit ficou bem próximo do lgbm, né?? Modelos lineares, quando bem usados, são extremamente poderosos!!
    
E notem que os scores na base teste tão bem próximos dos scores de CV, bem legal! ;)
</b></font>

<font color=blue><b>
Parabéns, pessoal! Espero que vocês tenha curtido a experiência do projeto e de todo esse tempo que passamos juntos!
    
Mais uma vez, desejo muuuito sucesso na vida e carreira de vocês! 
    
Valeu, galera -- e nos vemos no happy hour! ;)
</b></font>